# Saltwater Intrusion: case study for Ameland

<div>
<img src="./data/ameland.jpg" width="500"/>
<img src="./data/ameland_map.png" width="514">
</div>

This notebook is a conceptual 2D study of saltwater intrusion effects of hypothetical island with the topology of Ameland.
The stresses driving the simulation are freshwater recharge (RCH) from precipitation on the island, and the sea boundary 
that is accounted for using general head boundary cells (GHB). The location and values of boundary elements is given
with sealevel as a parameter, such that effects of sealevel rise can be studied.

In [ ]:
import flopy

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

In [ ]:
init_sealevel = -0.9
rise_per_period = 0.1

# load cross section from file:
# (data from https://www.ahn.nl/eerste-deel-van-ahn-5-is-beschikbaar
#  and cross section extracted from TIFs with QGIS)
arr = np.loadtxt("./data/ameland.dat")
x = arr[:,0]
y = arr[:,1]

# resample
length = np.amax(x) - np.amin(x)
nsamples = 41
xc = np.linspace(np.amin(x), np.amax(x), nsamples)
top = np.interp(xc, x, y)

In [ ]:
plt.figure(figsize=(12,3))
plt.plot(x, y, label="Ameland cross section")
plt.plot(xc, top, label="sampled TOP coordinates")
plt.hlines(init_sealevel, np.amin(x), np.amax(x), linestyles="dashed", color="grey", label="init. sea level")
plt.legend()

In [ ]:
nper = 21
year = 365.25
dtmin = 1.0
dt = 5 * year

duration = 500 * year # days
nsteps = duration / dt
period_data = [(duration, nsteps, 1.0)]

duration = 10 * year
nsteps = duration / dt
for iper in range(1,nper):
    period_data.append((duration, nsteps, 1.0))

In [ ]:
nlay = 16
nrow = 1
ncol = nsamples
delr = delc = length/ncol

topc = np.zeros((nlay, nrow, ncol), dtype=float)
botm = np.zeros((nlay, nrow, ncol), dtype=float)

dz = 160.0/nlay
topc[0] = top.reshape((nrow,ncol))
botm[0] = -5.0
for ilay in range(1, nlay):
    botm[ilay] = botm[ilay - 1] - dz
    topc[ilay] = topc[ilay - 1] - dz

In [ ]:
porosity = spec_yield = 0.35

conc_fresh = 0.0
conc_sea = 35.0
conc_limit = 20e-03 # 20 milligrams/liter for potable water

rch_rate = 0.001 # m/d

In [ ]:
def get_sea_level(iper):
  return init_sealevel + iper * rise_per_period

In [ ]:
def get_flooded_cols(sea_level):
  flooded_cols = []
  for icol in range(ncol):
      if sea_level > topc[0, 0, icol]:
          flooded_cols.append(icol)
      else: break
  for icol in reversed(range(ncol)):
      if sea_level > topc[0, 0, icol]:
          flooded_cols.append(icol)
      else: break
    
  return flooded_cols

In [ ]:
sim_name = "ameland"
gwt_name = "gwt"
gwf_name = "gwf"

sim = flopy.mf6.MFSimulation(
    sim_ws=sim_name,
    exe_name="mf6",
)

tdis = flopy.mf6.ModflowTdis(sim, 
                             nper=nper, 
                             perioddata=period_data)
ats_filerecord = sim_name + ".ats"
atsperiod = [
    (iper, dtmin, dtmin, dt, 2.0, 2.0) for iper in range(nper)
]
tdis.ats.initialize(
    maxats=len(atsperiod),
    perioddata=atsperiod,
    filename=ats_filerecord,
)

In [ ]:
imsgwf = flopy.mf6.ModflowIms(
    sim,
    complexity="complex",
    print_option="SUMMARY",
    linear_acceleration="bicgstab",
    outer_maximum=100,
    inner_maximum=100,
    outer_dvclose=1e-4,
    inner_dvclose=1e-5,
)

gwf = flopy.mf6.ModflowGwf(
    sim,
    modelname=gwf_name,
    print_input=False,
    save_flows=True, 
    newtonoptions="NEWTON UNDER_RELAXATION",
)

dis = flopy.mf6.ModflowGwfdis(
    gwf,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=topc[0],
    botm=botm,
    idomain=1,
    filename=f"{gwf_name}.dis",
)

strt = init_sealevel
ic = flopy.mf6.ModflowGwfic(gwf, strt=strt)

hk = 10.0
npf = flopy.mf6.ModflowGwfnpf(
    gwf,
    save_specific_discharge=True,
    icelltype=1,
    k=hk,
)

pd = [(0, 0.7, 0.0, gwt_name, "concentration")]
flopy.mf6.ModflowGwfbuy(
    gwf, 
    packagedata=pd
)

sto = flopy.mf6.ModflowGwfsto(
    gwf,
    save_flows=True,
    iconvert=1,
    ss=0.00001,
    sy=spec_yield,
    steady_state={0 : False},
    transient={0 : True},
)

rch_pkg = "rch_package"
rch_spd = {}
for iper in range(nper):
    sea_level = get_sea_level(iper)
    flooded_cols = get_flooded_cols(sea_level)
    recharge = np.ones((nrow,ncol), dtype=float) * rch_rate
    recharge[0, flooded_cols] = 0.0
    rch_spd[iper] = recharge
    
rch = flopy.mf6.ModflowGwfrcha(
    gwf,
    recharge=rch_spd,
    auxiliary="CONCENTRATION",
    aux={0 : conc_fresh},
    pname=rch_pkg,
)


ghb_pkg = "ghb_package"
ghb_spd = {}
for iper in range(nper):

    ghb_data = []
    sea_level = get_sea_level(iper)
    flooded_cols = get_flooded_cols(sea_level)

    # top GHBs
    for icol in flooded_cols:
        elev = topc[0, 0, icol] # top elevation
        area = delr * delc
        cell_height = topc[0, 0, icol] - botm[0, 0, icol]
        cond = hk * area / (0.5 * cell_height)
        ghb_data.append([(0, 0, icol), sea_level, cond, conc_sea, elev])
    # side GHBs
    for ilay in range(nlay):
        elev = 0.5 * (topc[ilay, 0, 0] + botm[ilay, 0, 0]) # node elevation
        area = delc * (topc[ilay, 0, 0] - botm[ilay, 0, 0])
        cond = hk * area / (0.5 * delr)
        ghb_data.append([(ilay, 0, 0), sea_level, cond, conc_sea, elev])
    for ilay in range(nlay):
        elev = 0.5 * (topc[ilay, 0, 0] + botm[ilay, 0, 0]) # node elevation
        area = delc * (topc[ilay, 0, ncol - 1] - botm[ilay, 0, ncol - 1])
        cond = hk * area / (0.5 * delr)
        ghb_data.append([(ilay, 0, ncol - 1), sea_level, cond, conc_sea, elev])
    
    ghb_spd[iper] = ghb_data

ghb = flopy.mf6.ModflowGwfghb(
    gwf,
    maxbound=len(ghb_spd),
    auxiliary=["CONCENTRATION","ELEVATION"],
    stress_period_data=ghb_spd,
    pname=ghb_pkg,
)

drn_pkg = "drn_package"
drn_spd = {}
for iper in range(nper):
    drn_data = []
    sea_level = get_sea_level(iper)
    flooded_cols = get_flooded_cols(sea_level)

    for icol in range(ncol):
        if icol in flooded_cols: continue
        cell_height = topc[0, 0, icol] - botm[0, 0, icol]
        area = delr * delc
        cond = hk * area / (0.5 * cell_height)  # kv / b
        drn_data.append([(0, 0, icol), topc[0, 0, icol], cond])
    
    drn_spd[iper] = drn_data

drn = flopy.mf6.ModflowGwfdrn(
    gwf,
    maxbound=len(drn_spd),
    stress_period_data=drn_spd,
    pname=drn_pkg,
)

oc = flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=f"{gwf.name}.hds",
    budget_filerecord=f"{gwf.name}.cbc",
    saverecord=[("HEAD", "LAST"), ("BUDGET", "LAST")],
    printrecord=[("BUDGET", "LAST")],
)

sim.register_ims_package(imsgwf, [gwf_name])


In [ ]:
gwt = flopy.mf6.ModflowGwt(
        sim,
        modelname=gwt_name,
        print_input=False,
        save_flows=True,
)

dis = flopy.mf6.ModflowGwtdis(
    gwt,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    idomain=1,
    top=topc[0],
    botm=botm,
)

conc_start = conc_sea
ic = flopy.mf6.ModflowGwtic(gwt, strt=conc_start, filename=f"{gwt_name}.ic")

adv = flopy.mf6.ModflowGwtadv(
  gwt, 
  scheme="tvd",
  ats_percel=0.7,
  filename=f"{gwt_name}.adv")

diffc = 0.0
alphal = 0.1
dsp = flopy.mf6.ModflowGwtdsp(
    gwt,
    diffc=diffc,
    alh=alphal,
    alv=alphal,
    ath1=0.0,
    atv=0.0,
    filename=f"{gwt_name}.dsp",
)


mst = flopy.mf6.ModflowGwtmst(
    gwt, 
    porosity=porosity,
    filename=f"{gwt_name}.mst"
)

# sources
sourcerecarray = [
    (rch_pkg, "AUX", "CONCENTRATION"),
    (ghb_pkg, "AUX", "CONCENTRATION")
]
ssm = flopy.mf6.ModflowGwtssm(
    gwt, sources=sourcerecarray, filename=f"{gwt_name}.ssm"
)

# output control
oc = flopy.mf6.ModflowGwtoc(
    gwt,
    budget_filerecord=f"{gwt_name}.cbc",
    concentration_filerecord=f"{gwt_name}.ucn",
    saverecord=[("CONCENTRATION", "LAST"), ("BUDGET", "LAST")],
)

imsgwt = flopy.mf6.ModflowIms(
        sim,
        complexity="complex",
        print_option="SUMMARY",
        linear_acceleration="bicgstab",
        outer_maximum=1000,
        inner_maximum=100,
        outer_dvclose=1e-4,
        inner_dvclose=1e-5,
        filename=f"{gwt_name}.ims",
    )

sim.register_ims_package(imsgwt, [gwt_name])

In [ ]:
gwfgwt = flopy.mf6.ModflowGwfgwt(
    sim,
    exgtype="GWF6-GWT6",
    exgmnamea=gwf_name,
    exgmnameb=gwt_name,
    filename="gwfgwt.exg",
)

In [ ]:
sim.write_simulation()
success, text = sim.run_simulation()

In [ ]:
if not success:
  raise ValueError(f"Simulation failed")

In [ ]:
times = gwf.output.head().get_times()
head_first = gwf.output.head().get_data(totim=times[0])
head_last = gwf.output.head().get_data(totim=times[-1])

In [ ]:
wt_first = flopy.utils.postprocessing.get_water_table(head_first)
wt_last = flopy.utils.postprocessing.get_water_table(head_last)
fig = plt.figure(figsize=(14,3))
plt.plot(xc, wt_first, label="initial water table (period = 1)")
plt.plot(xc, wt_last, label="final water table (period = nper)")
plt.plot(xc, top, label="top elevation")
plt.grid()
plt.legend()

In [ ]:
# returns 'volume' in m^2 (volume per meter along the y-axis)
def get_potable_volume(conc, limit):
  potable_volume = 0.0
  for ilay in range(nlay):
    for icol in range(ncol):
        if conc[ilay, 0, icol] < limit:
          vol = porosity * delr * 1.0 * (topc[ilay, 0, icol] - botm[ilay, 0, icol])
          potable_volume += vol
  return potable_volume

In [ ]:
plot_periods = [0, 3, int(nper/2), nper - 1]
nr_plots = len(plot_periods)

fig, axs = plt.subplots(len(plot_periods), 2, figsize=(16, 4 * nr_plots))
for idx, iper in enumerate(plot_periods):
  head = gwf.output.head().get_data(totim=times[iper])
  conc = gwt.output.concentration().get_data(totim=times[iper])

  # plot heads and BCs
  ax = axs[idx, 0]
  t_yrs = int(0.1 + times[iper]/365.25)
  csl_m = get_sea_level(iper) - init_sealevel
  ax.set_title(f"t = {t_yrs} years, sealevel rise = {csl_m:.1f}")
  pxs = flopy.plot.PlotCrossSection(model=gwf, line={"row": 0}, ax=ax)
  pa = pxs.plot_array(head, head=head)
  pxs.plot_grid()
  pc_ghb = pxs.plot_bc("GHB", ec="darkred", fc='none', lw=1.5, label="GHB", kper=iper)
  pxs.plot_bc("DRN", ec="darkblue", fc='none', lw=1.5,label="DRN", kper=iper)
  handles = [Polygon([(0,0), (10,0), (0,-10)], fc='none', color="darkred", label="GHB"),
             Polygon([(0,0), (10,0), (0,-10)], fc='none', color="darkblue", label="DRN")]
  cb = plt.colorbar(pa)
  cb.set_label("freshwater head [m]")
  ax.set_ylim(-120,)
  if idx == 0:
    ax.legend(handles=handles, loc="lower right", facecolor='lightgrey', framealpha=1)

  # plot concentration and contours
  ax = axs[idx, 1]
  ax.set_title(f"potable vol. = {int(get_potable_volume(conc, conc_limit))} m3/m")
  pxs = flopy.plot.PlotCrossSection(model=gwt, line={"row": 0}, ax=ax)
  pa = pxs.plot_array(conc, head=head)
  nr_levels = 5
  levels = np.linspace(conc_limit, conc_sea/2.0, nr_levels)
  cs = pxs.contour_array(conc, levels=levels, cmap="Reds")
  handles, labels = cs.legend_elements()
  pxs.plot_grid()
  cb = plt.colorbar(pa)
  cb.set_label("concentration [kg/m3]")
  ax.set_ylim(-120,)
  if idx == 0:
    ax.legend(handles, 
                    [f"{levels[i]:.2f}" for i in range(nr_levels)], 
                    loc='lower right',facecolor='lightgrey', framealpha=1)
